# ahahah

In [5]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
#edge_index = torch.tensor([[0, 1, 1],
#                           [1, 0, 2]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

print(data)

print(f"Data has been valutated as: {data.validate(raise_on_error=True)}")

Data(x=[3, 1], edge_index=[2, 3])
Data has been valutated as: True


In [6]:
next(iter(data))
for key, item in data:
    print(f'{key} found in data')
data.num_nodes

data.num_edges
data.num_node_features
data.has_isolated_nodes()
data.has_self_loops()
data.is_directed()

device = torch.device('cuda')
data = data.to(device)



x found in data
edge_index found in data


True

## Use Dataset

In [7]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...


600
6
3


Done!


## Splitting dataset in train and test

In [ ]:

dataset = dataset.shuffle()
# or 
# perm = torch.randperm(len(dataset))
#dataset = dataset[perm]
train_dataset = dataset[:540]
test_dataset = dataset[540:]


### dataset cora

In [20]:
from torch_geometric.datasets import Planetoid

dataset2 = Planetoid(root='/tmp/Cora', name='Cora')

print(f"The lenght of the dataset is : {len(dataset2)}")

print(f"Each node has {dataset2.num_node_features} features")

#data = dataset[0]
#data.is_undirected()
#dataTrain = data.train_mask.sum().item()
#dataVal = data.val_mask.sum().item()
#dataTest = data.test_mask.sum().item()



The lenght of the dataset is : 1
Each node has 1433 features


In [21]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import scatter

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)

for data in loader:
    print(data)
    print(data.num_graphs)
    x = scatter(data.x, data.batch, dim=0, reduce='mean')
    print(x.size())



DataBatch(edge_index=[2, 4358], x=[1125, 21], y=[32], batch=[1125], ptr=[33])
32
DataBatch(edge_index=[2, 3480], x=[884, 21], y=[32], batch=[884], ptr=[33])
32
DataBatch(edge_index=[2, 3784], x=[956, 21], y=[32], batch=[956], ptr=[33])
32
DataBatch(edge_index=[2, 3898], x=[1001, 21], y=[32], batch=[1001], ptr=[33])
32
DataBatch(edge_index=[2, 4226], x=[1081, 21], y=[32], batch=[1081], ptr=[33])
32
DataBatch(edge_index=[2, 4536], x=[1203, 21], y=[32], batch=[1203], ptr=[33])
32
DataBatch(edge_index=[2, 3862], x=[1058, 21], y=[32], batch=[1058], ptr=[33])
32
DataBatch(edge_index=[2, 4008], x=[1057, 21], y=[32], batch=[1057], ptr=[33])
32
DataBatch(edge_index=[2, 3772], x=[967, 21], y=[32], batch=[967], ptr=[33])
32
DataBatch(edge_index=[2, 3892], x=[1093, 21], y=[32], batch=[1093], ptr=[33])
32
DataBatch(edge_index=[2, 3692], x=[990, 21], y=[32], batch=[990], ptr=[33])
32
DataBatch(edge_index=[2, 4248], x=[1070, 21], y=[32], batch=[1070], ptr=[33])
32
DataBatch(edge_index=[2, 4536], x=[1

## Data Transform

Transforms are a common way in torchvision to transform images and perform augmentation. PyG comes with its own transforms, which expect a Data object as input and return a new transformed Data object. Transforms can be chained together using torch_geometric.transforms.Compose and are applied before saving a processed dataset on disk (pre_transform) or before accessing a graph in a dataset (transform).

Let’s look at an example, where we apply transforms on the ShapeNet dataset (containing 17,000 3D shape point clouds and per point labels from 16 shape categories).

In [22]:
from torch_geometric.datasets import ShapeNet
import torch_geometric.transforms as T

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

print(f"Standard dataset {dataset[0]}")

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

print(f"Augmented dataset {dataset[0]}")

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...


Standard dataset Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])
Augmented dataset Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])


Done!
/Users/beppe2hd/miniforge3/envs/GNNtestEnv/lib/python3.10/site-packages/torch_geometric/data/dataset.py:209: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to delete '/tmp/ShapeNet/processed' first
  warnings.warn(


## Learning Methods on Graphs

In [28]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

dataset = Planetoid(root='/tmp/Cora', name='Cora')

print(dataset[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [31]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T

dataset = Planetoid(root='/tmp/Cora', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
class GCNExtra(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, 6)
        self.conv3 = GCNConv(6, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        x = F.log_softmax(x, dim=1)

        return x
    
print(dataset)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("First model")

model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')


print("Second model")

model = GCNExtra().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Cora()
First model
Accuracy: 0.8050
Second model
Accuracy: 0.7850
